In [1]:
import random

import torchvision.models as models
import torch.nn as nn
import pandas as pd
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torchvision.models as models

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-02-23 02:44:34.583118: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 02:44:34.630236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following ins

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'

In [3]:
#TODO Возможно нужно подбирать гиперпаораметры 
IMG_SIZE = 255
BATCH_SIZE = 4
EPOCHS = 100
SEQ_LENGTH = 100

In [4]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])


In [5]:
class CustomDataset(Dataset):
    def __init__(self, root_path="", df_path="train.csv", img_size=224, SEQ_LENGTH=100, transform=None):
        self.SEQ_LENGTH = SEQ_LENGTH
        self.root_path = root_path
        self.img_size = img_size
        df = pd.read_csv(df_path)
        self.video_paths = df['path'].tolist()
        self.labels = df['label'].tolist()
        self.transform = transform
        unique_labels = sorted(set(self.labels))
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
        self.idx_to_label = {idx: label for label, idx in self.label_to_idx.items()}

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.video_paths[idx])
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        frames = []
        if total_frames >= self.SEQ_LENGTH:
            frame_indices = np.linspace(0, total_frames - 1, self.SEQ_LENGTH, dtype=int)
        else:
            frame_indices = np.tile(np.arange(total_frames), self.SEQ_LENGTH // total_frames + 1)[:self.SEQ_LENGTH]

        for i in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            frame_tensor = self.transform(frame).unsqueeze(0).to(device)
            frames.append(frame_tensor)

        while len(frames) < self.SEQ_LENGTH:
            frames.append(torch.zeros_like(frames[0]))

        frames_tensor = torch.cat(frames, dim=0)
        frames_tensor = frames_tensor.permute(1,0,2,3)
        return frames_tensor, self.label_to_idx[self.labels[idx]]

In [6]:
train_dataset = CustomDataset(df_path="train.csv", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_dataset = CustomDataset(df_path="test.csv", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [7]:
import torch
import torchvision

# Загрузка предварительно обученной модели 3D ResNet
#model = torchvision.models.video.r3d_18(num_classes=1).to(device)
import torch.nn.functional as F

# Пример модификации модели
class ModifiedModel(nn.Module):
    def __init__(self):
        super(ModifiedModel, self).__init__()
        self.base_model = torchvision.models.video.r3d_18(num_classes=512)  # Подставьте свою модель
        self.fc = nn.Linear(512, 1)  # Пример линейного слоя с одним выходом

    def forward(self, x):
        x = self.base_model(x)
        x = F.sigmoid(self.fc(x))  # Применяем сигмоиду к выходу
        return x
model = ModifiedModel().to(device)

In [8]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
import logging

# Конфигурация логирования
logging.basicConfig(filename='hight.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Переменная для хранения наивысшей точности
best_accuracy = 0.0

# Цикл обучения
for epoch in range(EPOCHS):
    model.train()  # Переводим модель в режим обучения
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Обнуляем градиенты параметров
        optimizer.zero_grad()

        # Прямой проход: вычисляем предсказанные значения
        outputs = model(data)

        # Вычисляем потери
        loss = criterion(outputs, target.float().view(-1, 1))

        # Обратное распространение и оптимизация
        loss.backward()
        optimizer.step()

        # Считаем общие потери
        running_loss += loss.item()

        # Выводим промежуточные результаты в терминал
        if batch_idx % 10 == 9:  # Выводим каждые 100 батчей
            print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}/{len(train_loader)}, Loss: {running_loss / 100:.4f}')
            logging.info(f'Epoch {epoch + 1}, Batch {batch_idx + 1}/{len(train_loader)}, Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

    # Оценка модели после каждой эпохи
    model.eval()  # Переводим модель в режим оценки
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            test_loss += criterion(outputs, target.float().view(-1, 1)).item()
            predicted = torch.round(outputs)
            total += target.size(0)
            correct += (predicted == target.float().view(-1, 1)).sum().item()

    accuracy = (correct / total) * 100
    print(f'Test Loss: {test_loss / len(test_loader):.4f}, Accuracy: {accuracy:.2f}%')
    logging.info(f'Test Loss: {test_loss / len(test_loader):.4f}, Accuracy: {accuracy:.2f}%')

    # Сохраняем модель, если достигнута лучшая точность
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), '256_best_model.pth')
        logging.info('Best model saved.')

logging.info('Training complete.')


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.91 GiB. GPU 0 has a total capacity of 14.58 GiB of which 607.56 MiB is free. Process 16416 has 13.98 GiB memory in use. Of the allocated memory 13.84 GiB is allocated by PyTorch, and 14.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
device